## Exploring Data with Dataframes and Spark SQL
In this exercise, you will explore data using the Spark Dataframe API and Spark SQL.
https://microsoftlearning.github.io/databricks-intro/

### Load Data Using an Explicit Schema
Now you can load the data into a dataframe. If the structure of the data is known ahead of time, you can explicitly specify the schema for the dataframe.

Modify the code below to reflect your Azure blob storage account name, and then click the &#9658; button at the top right of the cell to run it.

In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

path="/FileStore/tables/"

flightSchema = StructType([
  StructField("DayofMonth", IntegerType(), False),
  StructField("DayOfWeek", IntegerType(), False),
  StructField("Carrier", StringType(), False),
  StructField("OriginAirportID", IntegerType(), False),
  StructField("DestAirportID", IntegerType(), False),
  StructField("DepDelay", IntegerType(), False),
  StructField("ArrDelay", IntegerType(), False),
])


flights = spark.read.csv(path+"raw_flight_data-eb9f8.csv", schema=flightSchema, header=True)
flights.show()

+----------+---------+-------+---------------+-------------+--------+--------+
DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
 19| 5| DL| 11433| 13303| -3| 1|
 19| 5| DL| 14869| 12478| 0| -8|
 19| 5| DL| 14057| 14869| -4| -15|
 19| 5| DL| 15016| 11433| 28| 24|
 19| 5| DL| 11193| 12892| -6| -11|
 19| 5| DL| 10397| 15016| -1| -19|
 19| 5| DL| 15016| 10397| 0| -1|
 19| 5| DL| 10397| 14869| 15| 24|
 19| 5| DL| 10397| 10423| 33| 34|
 19| 5| DL| 11278| 10397| 323| 322|
 19| 5| DL| 14107| 13487| -7| -13|
 19| 5| DL| 11433| 11298| 22| 41|
 19| 5| DL| 11298| 11433| 40| 20|
 19| 5| DL| 11433| 12892| -2| -7|
 19| 5| DL| 10397| 12451| 71| 75|
 19| 5| DL| 12451| 10397| 75| 57|
 19| 5| DL| 12953| 10397| -1| 10|
 19| 5| DL| 11433| 12953| -3| -10|
 19| 5| DL| 10397| 14771| 31| 38|
 19| 5| DL| 13204| 10397| 8| 25|
+----------+---------+-------+---------------+-------------+--------+--------+
only showing top 20 rows

### Infer a Data Schema
If the structure of the data source is unknown, you can have Spark automatically infer the schema.

In this case, you will load data about airports without knowing the schema.

Modify the code below to reflect your Azure blob storage account name, and then run the cell.

In [5]:
airports = spark.read.csv(path+"airports.csv", header=True, inferSchema=True)
airports.show()

+----------+-----------+-----+--------------------+
airport_id| city|state| name|
+----------+-----------+-----+--------------------+
 10165|Adak Island| AK| Adak|
 10299| Anchorage| AK|Ted Stevens Ancho...|
 10304| Aniak| AK| Aniak Airport|
 10754| Barrow| AK|Wiley Post/Will R...|
 10551| Bethel| AK| Bethel Airport|
 10926| Cordova| AK|Merle K Mudhole S...|
 14709| Deadhorse| AK| Deadhorse Airport|
 11336| Dillingham| AK| Dillingham Airport|
 11630| Fairbanks| AK|Fairbanks Interna...|
 11997| Gustavus| AK| Gustavus Airport|
 12523| Juneau| AK|Juneau International|
 12819| Ketchikan| AK|Ketchikan Interna...|
 10245|King Salmon| AK| King Salmon Airport|
 10170| Kodiak| AK| Kodiak Airport|
 13970| Kotzebue| AK| Ralph Wien Memorial|
 13873| Nome| AK| Nome Airport|
 14256| Petersburg| AK|Petersburg James ...|
 14828| Sitka| AK|Sitka Rocky Gutie...|
 12807| St. Mary's| AK| St. Mary's Airport|
 11445| Unalaska| AK| Unalaska Airport|
+----------+-----------+-----+--------------------+
only showing top 20 rows

### Use Dataframe Methods
Spark DataFrames provide functions that you can use to extract and manipulate data. For example, you can use the **select** function to return a new dataframe containing columns selected from an existing dataframe.

In [7]:
cities = airports.select("city", "name")
cities.show()

+-----------+--------------------+
 city| name|
+-----------+--------------------+
Adak Island| Adak|
 Anchorage|Ted Stevens Ancho...|
 Aniak| Aniak Airport|
 Barrow|Wiley Post/Will R...|
 Bethel| Bethel Airport|
 Cordova|Merle K Mudhole S...|
 Deadhorse| Deadhorse Airport|
 Dillingham| Dillingham Airport|
 Fairbanks|Fairbanks Interna...|
 Gustavus| Gustavus Airport|
 Juneau|Juneau International|
 Ketchikan|Ketchikan Interna...|
King Salmon| King Salmon Airport|
 Kodiak| Kodiak Airport|
 Kotzebue| Ralph Wien Memorial|
 Nome| Nome Airport|
 Petersburg|Petersburg James ...|
 Sitka|Sitka Rocky Gutie...|
 St. Mary's| St. Mary's Airport|
 Unalaska| Unalaska Airport|
+-----------+--------------------+
only showing top 20 rows

### Combine Operations
You can combine functions in a single statement to perform multiple operations on a dataframe. In this case, you will use the **join** function to combine the **flights** and **airports** dataframes, and then use the **groupBy** and **count** functions to return the number of flights from each airport.

In [9]:
flightsByOrigin = flights.join(airports, flights.OriginAirportID == airports.airport_id).groupBy("city").count()
flightsByOrigin.show()

+-----------------+------+
 city| count|
+-----------------+------+
 Phoenix| 90281|
 Omaha| 13537|
 Raleigh/Durham| 28436|
 Anchorage| 7777|
 Dallas| 19503|
 Oakland| 25503|
 San Antonio| 23090|
 Philadelphia| 47659|
 Louisville| 10953|
Dallas/Fort Worth|105024|
 Los Angeles|118684|
 Sacramento| 25193|
 Indianapolis| 18099|
 Cleveland| 25261|
 San Diego| 45783|
 San Francisco| 84675|
 Nashville| 34927|
 Oklahoma City| 13967|
 Detroit| 62879|
 Portland| 30640|
+-----------------+------+
only showing top 20 rows

### Count the Rows in a Dataframe
Now that you're familiar with working with dataframes, a key task when building predictive solutions is to explore the data, determing statistics that will help you understand the data before building predictive models. For example, how many rows of flight data do you actually have?

In [11]:
flights.count()

Out[ 7 ]: 2719418

### Determine the Presence of Duplicates
The data you have to work with won't always be perfect - often you'll want to *clean* the data; for example to detect and remove duplicates that might affect your model. You can use the **dropDuplicates** function to create a new dataframe with the duplicates removed, enabling you to determine how many rows are duplicates of other rows.

In [13]:
flights.count() - flights.dropDuplicates().count()

Out[ 8 ]: 22435

### Identify Missing Values
As well as determining if duplicates exist in your data, you should detect missing values, and either remove rows containing missing data or replace the missing values with a suitable relacement. The **dropna** function creates a dataframe with any rows containing missing data removed - you can specify a subset of columns, and whether the row should be removed in *any* or *all* values are missing. You can then use this new dataframe to determine how many rows contain missing values.

In [15]:
flights.count() - flights.dropDuplicates().dropna(how="any", subset=["ArrDelay", "DepDelay"]).count()

Out[ 9 ]: 46233

### Clean the Data
Now that you've identified that there are duplicates and missing values, you can clean the data by removing the duplicates and replacing the missing values. The **fillna** function replaces missing values with a specified replacement value. In this case, you'll remove all duplicate rows and replace missing **ArrDelay** and **DepDelay** values with **0**.

In [17]:
data=flights.dropDuplicates().fillna(value=0, subset=["ArrDelay", "DepDelay"])
data.count()

Out[ 10 ]: 2696983

## Explore the Data
Now that you've cleaned the data, you can start to explore it and perform some basic analysis. Let's start by examining the lateness of a flight. The dataset includes the **ArrDelay** field, which tells you how many minutes behind schedule a flight arrived. However, if a flight is only a few minutes behind schedule, you might not consider it *late*. Let's make our definition of lateness such that flights that arrive within 25 minutes of their scheduled arrival time are considered on-time, but any flights that are more than 25 minutes behind schedule are classified as *late*. We'll add a column to indicate this classification:

In [19]:
data = data.select("DayofMonth", "DayOfWeek", "Carrier", "OriginAirportID","DestAirportID",
                   "DepDelay", "ArrDelay", ((col("ArrDelay") > 25).cast("Int").alias("Late")))
data.show()

+----------+---------+-------+---------------+-------------+--------+--------+----+
DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|Late|
+----------+---------+-------+---------------+-------------+--------+--------+----+
 19| 5| DL| 12889| 13487| 7| 16| 0|
 18| 4| DL| 12892| 13487| 5| -4| 0|
 18| 4| DL| 14635| 11193| -7| -12| 0|
 18| 4| DL| 10397| 11298| -3| 44| 1|
 18| 4| DL| 10397| 12451| 1| -7| 0|
 19| 5| DL| 13487| 11433| -3| -21| 0|
 19| 5| DL| 14771| 13487| 340| 325| 1|
 19| 5| DL| 10397| 11298| 20| 20| 0|
 19| 5| DL| 12953| 13487| 4| -28| 0|
 19| 5| DL| 14869| 11278| -4| -24| 0|
 20| 6| DL| 12889| 12478| -6| -9| 0|
 16| 2| DL| 10397| 11193| -4| -19| 0|
 16| 2| DL| 14747| 13487| -16| -17| 0|
 16| 2| DL| 10721| 10397| 9| 3| 0|
 17| 3| DL| 12173| 14747| 10| 8| 0|
 17| 3| DL| 14869| 14771| 0| 0| 0|
 18| 4| DL| 10397| 13495| 0| 6| 0|
 18| 4| DL| 15304| 11433| -7| -13| 0|
 18| 4| DL| 10397| 11618| -1| -19| 0|
 18| 4| DL| 13487| 10397| 3| -3| 0|
+----------+---------+-------+---------------+-------------+--------+--------+----+
only showing top 20 rows

### Explore Summary Statistics and Data Distribution
Predictive modeling is based on statistics and probability, so we should take a look at the summary statistics for the columns in our data. The **describe** function returns a dataframe containing the **count**, **mean**, **standard deviation**, **minimum**, and **maximum** values for each numeric column.

In [21]:
data.describe().show()

+-------+------------------+-----------------+-------+------------------+------------------+------------------+------------------+-------------------+
summary| DayofMonth| DayOfWeek|Carrier| OriginAirportID| DestAirportID| DepDelay| ArrDelay| Late|
+-------+------------------+-----------------+-------+------------------+------------------+------------------+------------------+-------------------+
 count| 2696983| 2696983|2696983| 2696983| 2696983| 2696983| 2696983| 2696983|
 mean|15.798996508320593|3.900369412784582| null|12742.459424846207| 12742.85937657004|10.531134234068217|6.6679285705545785|0.14104983234970336|
 stddev| 8.80126719913545|1.986458242170198| null|1502.0359941370625|1501.9939589817984| 36.06172819056572|38.583861473580725|0.34807301255078194|
 min| 1| 1| 9E| 10140| 10140| -63| -94| 0|
 max| 31| 7| YV| 15376| 15376| 1863| 1845| 1|
+-------+------------------+-----------------+-------+------------------+------------------+------------------+------------------+-------------------+

The *DayofMonth* is a value between 1 and 31, and the mean is around halfway between these values; which seems about right. The same is true for the *DayofWeek* which is a value between 1 and 7. *Carrier* is a string, so there are no numeric statistics; and we can ignore the statistics for the airport IDs - they're just unique identifiers for the airports, not actually numeric values. The departure and arrival delays range between 63 or 94 minutes ahead of schedule, and over 1,800 minutes behind schedule. The means are much closer to zero than this, and the standard deviation is quite large; so there's quite a bit of variance in the delays. The *Late* indicator is a 1 or a 0, but the mean is very close to 0; which implies that there significantly fewer late flights than non-late flights.

Let's verify that assumption by creating a table and using the **Spark SQL** API to run a SQL statement that counts the number of late and non-late flights:

In [23]:
data.createOrReplaceTempView("flightData")
spark.sql("SELECT Late, COUNT(*) AS Count FROM flightData GROUP BY Late").show()

+----+-------+
Late| Count|
+----+-------+
 1| 380409|
 0|2316574|
+----+-------+

Yes, it looks like there are significantly more non-late flights than late ones - we can see this more clearly with a visualization, so let's use the inline **%sql** magic to query the table and bring back some results we can display as a chart:

In [25]:
%sql
SELECT * FROM flightData

DayofMonth DayOfWeek Carrier OriginAirportID DestAirportID DepDelay ArrDelay Late 19 5 DL 12889 13487 7 16 0 18 4 DL 12892 13487 5 -4 0 18 4 DL 14635 11193 -7 -12 0 18 4 DL 10397 11298 -3 44 1 18 4 DL 10397 12451 1 -7 0 19 5 DL 13487 11433 -3 -21 0 19 5 DL 14771 13487 340 325 1 19 5 DL 10397 11298 20 20 0 19 5 DL 12953 13487 4 -28 0 19 5 DL 14869 11278 -4 -24 0 20 6 DL 12889 12478 -6 -9 0 16 2 DL 10397 11193 -4 -19 0 16 2 DL 14747 13487 -16 -17 0 16 2 DL 10721 10397 9 3 0 17 3 DL 12173 14747 10 8 0 17 3 DL 14869 14771 0 0 0 18 4 DL 10397 13495 0 6 0 18 4 DL 15304 11433 -7 -13 0 18 4 DL 10397 11618 -1 -19 0 18 4 DL 13487 10397 3 -3 0 20 6 DL 12892 10397 -3 -14 0 20 6 DL 10397 10721 5 -19 0 20 6 DL 10721 11433 -5 -16 0 20 6 DL 10693 10397 -2 -6 0 20 6 DL 13204 10397 -1 -14 0 20 6 DL 12892 11433 5 -3 0 20 6 DL 11298 10397 -5 -15 0 20 6 DL 12953 11697 -3 -2 0 21 7 DL 14027 11433 1 -21 0 21 7 DL 14730 10397 -5 -7 0 21 7 DL 14057 14869 -7 -13 0 22 1 DL 13487 11433 0 -5 0 20 6 DL 13495 10397 -3 -19 0 20 6 DL 13487 11278 -7 -22 0 20 6 DL 14869 11433 -4 -14 0 21 7 DL 10397 14843 0 -20 0 21 7 DL 10397 13244 -5 7 0 21 7 DL 10397 13342 1 -16 0 21 7 DL 11433 11697 4 3 0 21 7 DL 12478 14057 58 31 1 21 7 DL 15304 11193 -7 -19 0 23 2 DL 10397 11193 -1 -7 0 23 2 DL 15304 12478 20 23 0 17 3 DL 10397 12191 -6 -4 0 17 3 DL 14027 12953 -4 -12 0 18 4 DL 14100 13487 262 268 1 18 4 DL 10397 14683 27 29 1 18 4 DL 11433 14107 -6 -9 0 18 4 DL 11298 10397 43 69 1 23 2 DL 14771 12478 -5 -13 0 23 2 DL 14492 12892 2 -13 0 22 1 DL 10397 10821 -2 -25 0 22 1 DL 10397 11618 -2 -2 0 22 1 DL 14869 14679 209 201 1 23 2 DL 10397 13204 -4 -9 0 24 3 DL 11433 12339 16 3 0 24 3 DL 13930 10397 23 23 0 24 3 DL 12892 13204 3 -13 0 23 2 DL 11433 13198 -4 4 0 23 2 DL 14747 14869 -5 -24 0 24 3 DL 13204 10397 -3 -11 0 24 3 DL 12892 13487 0 -7 0 24 3 DL 11433 13204 -4 -8 0 24 3 DL 12339 10397 -7 -12 0 24 3 DL 10397 14122 -1 -17 0 24 3 DL 15016 13487 110 104 1 24 3 DL 13487 14107 -7 7 0 26 5 DL 14893 10397 -3 -1 0 23 2 DL 13303 12953 -3 -16 0 23 2 DL 12889 11193 -6 -35 0 25 4 DL 11066 10397 -1 -7 0 25 4 DL 13487 12892 107 108 1 25 4 DL 14747 12478 15 11 0 25 4 DL 12339 10397 -3 -4 0 25 4 DL 14100 13487 -3 -14 0 26 5 DL 13342 11433 -3 -19 0 24 3 DL 14492 10397 -2 -12 0 24 3 DL 10397 13851 -1 -10 0 24 3 DL 10397 13495 -1 -5 0 24 3 DL 10397 14730 -3 -7 0 25 4 DL 13487 12889 -6 -25 0 25 4 DL 14107 14869 6 3 0 25 4 DL 11433 14100 -1 2 0 25 4 DL 14679 10397 -1 -16 0 25 4 DL 14524 10397 -6 -13 0 25 4 DL 13487 14771 4 -22 0 25 4 DL 11433 13204 -7 -23 0 25 4 DL 10397 11057 -3 -19 0 25 4 AA 12953 13303 -3 -6 0 20 6 AA 11298 12889 10 13 0 2 2 AA 13930 12892 2 -8 0 26 5 DL 11057 10397 -6 -9 0 26 5 DL 10397 13495 -1 -20 0 26 5 DL 11433 13930 -4 -20 0 26 5 DL 12478 13303 1 -3 0 27 6 DL 12478 12889 -5 -45 0 27 6 DL 14679 10397 -6 -12 0 27 6 DL 12953 13303 -6 -18 0 27 6 DL 13198 10397 -7 -12 0 26 5 DL 10821 10397 0 0 0 26 5 DL 10397 12892 0 -14 0 27 6 DL 14747 10397 -5 -8 0 27 6 DL 14492 11433 -5 -17 0 27 6 DL 10397 12451 -5 -9 0 27 6 DL 13342 11433 -1 -18 0 28 7 DL 11057 11433 -5 -17 0 28 7 DL 11433 15304 1 5 0 28 7 DL 12953 15304 -7 -3 0 28 7 DL 10397 10792 -2 -21 0 28 7 DL 12478 12889 -3 -30 0 29 1 DL 14679 10397 -10 -4 0 30 2 DL 10397 11193 -3 -16 0 30 2 DL 14107 11433 -4 -1 0 28 7 DL 12892 12478 -3 -4 0 29 1 DL 10397 12266 -3 -21 0 29 1 DL 12889 14869 715 716 1 29 1 DL 15304 12953 43 33 1 29 1 DL 10721 12478 -7 -30 0 29 1 DL 11618 10397 61 46 1 29 1 DL 10397 15376 17 -15 0 29 1 DL 13204 12892 -1 -11 0 29 1 DL 11433 10397 -1 -13 0 30 2 DL 11066 10397 -2 -4 0 30 2 DL 14679 10397 -12 -6 0 28 7 DL 10397 13303 11 -8 0 28 7 DL 12451 10397 35 44 1 29 1 DL 13204 10397 38 48 1 29 1 DL 10397 10821 -4 -4 0 29 1 DL 11278 11433 -7 -18 0 29 1 DL 11433 13244 109 95 1 29 1 DL 13232 10397 -11 -17 0 30 2 DL 13303 10397 -4 4 0 26 5 DL 11697 10397 -5 -6 0 26 5 DL 10397 12889 0 -16 0 4 4 EV 11066 12266 -11 -11 0 30 2 DL 11433 10397 -5 -15 0 30 2 DL 12953 14027 -3 -16 0 15 1 EV 12266 14683 -4 2 0 20 6 EV 11042

The query returns a table of data containing the first 1000 rows, which should be a big enough sample for us to explore. To see the distribution of *Late* classes (1 for late, 0 for on-time), in the visualization drop-down list under the table above, click **Bar**. Then click **Plot Options** and configure the visualization like this:
- **Keys**: Late
- **Series Groupings**: *none*
- **Values**: &lt;id&gt;
- **Aggregation**: Count
- **Display type**: Bar chart
- **Grouped**: Selected

You should be able to see that the sample includes significantly more on-time flights than late ones. This indicates that the dataset is *imbalanced*; which might adversely affect the accuracy of any machine learning model we train from this data.

Additionally, you observed earlier that there are some extremely high **DepDelay** and **ArrDelay** values that might be skewing the distribution of the data disproportionately because of a few *outliers*. Let's visualize the distribution of these columns to explore this. Change the **Plot Options** settings as follows:
- **Keys**: *none*
- **Series Groupings**: *none*
- **Values**: DepDelay
- **Aggregation**: Count
- **Display Type**: Histogram plot
- **Number of bins**: 20

You can drag the handle at the bottom right of the visualization to resize it. Note that the data is skewed such that most flights have a **DepDelay** value within 100 or so minutes of 0. However, there are a few flights with extremely high delays. Another way to view this distribution is a *box plot*. Change the **Plot Options** as follows:
- **Keys**: *none*
- **Series Groupings**: *none*
- **Values**: DepDelay
- **Aggregation**: Count
- **Display Type**: Box plot

The box plot consists of a box with a line indicating the median departure delay, and *whiskers* extending from the box to show the first and fourth quartiles of the data, with statistical *outliers* shown as small circles. This confirms the extremely skewed distribution of **DepDelay** values seen in the histogram (and if you care to check, you'll find that the **ArrDelay** column has a similar distribution).

Let's address the outliers and imbalanced classes in our data by removing rows with extreme delay values, and *undersampling* the more common on-time flights:

In [27]:
from pyspark.sql.functions import rand

# Remove outliers - let's make the cut-off 150 minutes.
data = data.filter("DepDelay < 150 AND ArrDelay < 150")

# Separate the late and on-time flights
pos = data.filter("Late = 1")
neg = data.filter("Late = 0")

# undersample the most prevalent class to get a roughly even distribution
posCount = pos.count()
negCount = neg.count()
if posCount > negCount:
  pos = pos.sample(True, negCount/(negCount + posCount))
else:
  neg = neg.sample(True, posCount/(negCount + posCount))
  
# shuffle into random order (so a sample of the first 1000 has a mix of classes)
data = neg.union(pos).orderBy(rand())

# Replace the temporary table so we can query and visualize the balanced dataset
data.createOrReplaceTempView("flightData")

# Show the statistics
data.describe().show()

+-------+------------------+------------------+-------+------------------+------------------+------------------+------------------+------------------+
summary| DayofMonth| DayOfWeek|Carrier| OriginAirportID| DestAirportID| DepDelay| ArrDelay| Late|
+-------+------------------+------------------+-------+------------------+------------------+------------------+------------------+------------------+
 count| 637837| 637837| 637837| 637837| 637837| 637837| 637837| 637837|
 mean|15.828081469090066| 3.882261455512929| null|12711.577551945089|12743.568071466534|28.869026099144453|29.283014312434055|0.5340674811903354|
 stddev| 8.67119571411199|1.9603500416109694| null|1487.0891454643113|1499.4695305041769| 37.24794104542385| 39.38128251741398|0.4988384476521919|
 min| 1| 1| 9E| 10140| 10140| -45| -68| 0|
 max| 31| 7| YV| 15376| 15376| 149| 149| 1|
+-------+------------------+------------------+-------+------------------+------------------+------------------+------------------+------------------+

Now the maximums for the **DepDelay** and **ArrDelay** are clipped at under 150, and the mean value for the binary *Late* class is nearer 0.5; indicating a more or less even number of each class. We removed some data to accomplish this balancing act, but there are still a substantial number of rows for us to train a machine learning model with, and now the data is more balanced. Let's visualize the data again to confirm this:

In [29]:
%sql
SELECT * FROM flightData

DayofMonth DayOfWeek Carrier OriginAirportID DestAirportID DepDelay ArrDelay Late 25 2 DL 14869 11278 24 22 0 6 6 WN 11259 10140 42 35 1 13 2 DL 14100 11433 31 31 1 31 6 DL 10397 15304 -4 -8 0 23 5 9E 11433 12478 -1 0 0 9 3 US 11057 13930 66 56 1 25 3 EV 13198 12264 -6 -16 0 26 4 WN 14107 11066 39 30 1 10 3 B6 10721 14843 127 115 1 17 3 UA 14771 14107 -3 -14 0 10 3 WN 11259 14683 68 84 1 5 3 UA 11618 13204 -3 0 0 25 4 EV 11618 12264 95 82 1 23 2 EV 11193 12266 85 90 1 17 3 US 14492 14100 115 108 1 29 1 OO 14831 12892 141 125 1 3 1 WN 13931 10821 8 5 0 28 7 MQ 13930 10529 45 31 1 23 4 WN 13232 10693 105 90 1 9 2 B6 15304 10721 -2 -19 0 5 1 AA 14107 11298 118 116 1 5 7 DL 10397 14771 50 56 1 9 5 UA 11618 13930 52 46 1 12 4 9E 11433 13931 0 0 0 31 6 F9 11292 13198 1 -6 0 18 6 WN 11292 12264 14 24 0 18 4 UA 10721 14771 113 89 1 27 5 B6 13204 12953 -6 -2 0 16 4 WN 14908 13796 42 31 1 21 7 WN 11292 13796 -5 -13 0 28 2 B6 14843 13204 50 30 1 6 1 B6 13495 12478 28 -1 0 18 5 WN 13232 12264 3 -5 0 3 3 DL 10397 13342 125 112 1 4 6 US 11057 14100 -2 -1 0 17 2 B6 10721 14524 -7 -19 0 13 2 US 11278 12953 31 56 1 21 3 FL 10397 12191 9 -1 0 24 3 EV 10693 11618 79 59 1 4 3 WN 14107 14679 6 2 0 17 2 AA 13930 11298 1 -3 0 14 1 B6 10721 12264 78 64 1 28 2 9E 11433 12264 -4 -16 0 9 4 WN 15016 10693 0 2 0 24 5 AA 11298 13851 29 41 1 8 4 WN 14107 10140 33 24 0 21 5 AA 13930 12953 52 52 1 9 5 UA 13204 12266 47 33 1 26 5 WN 15016 11259 5 28 1 10 3 B6 10721 13204 5 84 1 24 3 WN 11066 14107 17 29 1 2 1 B6 12953 14635 143 146 1 18 2 DL 10397 15016 11 44 1 14 2 DL 10721 14869 -3 -17 0 19 4 UA 12889 13930 90 95 1 17 4 AA 14683 11298 94 76 1 12 5 UA 12953 13930 121 95 1 3 5 9E 13487 11042 -5 34 1 29 7 B6 14843 12478 71 97 1 30 7 B6 15304 10721 54 45 1 17 3 OO 14107 14771 -9 -21 0 30 2 FL 14635 13342 -5 -6 0 15 1 WN 14908 11292 75 117 1 12 5 AA 11057 11298 65 46 1 3 2 OO 13851 11292 -13 -10 0 26 4 WN 13232 10721 13 7 0 12 5 UA 13930 11618 78 56 1 3 2 HA 13830 12173 2 4 0 5 1 OO 14771 14570 85 76 1 3 4 UA 11292 12889 19 26 1 8 7 AA 13930 13303 22 -1 0 30 3 WN 14492 10693 -3 -2 0 7 7 UA 13930 14771 91 68 1 26 6 YV 14683 12264 -8 -31 0 18 4 B6 12889 12954 -11 -25 0 22 6 B6 14635 10721 -1 -7 0 3 6 F9 11292 14771 18 40 1 31 5 AA 13930 11298 20 39 1 3 2 YV 11057 12264 -6 -20 0 13 7 AS 14831 14057 -5 16 0 3 3 OO 14831 12892 -7 -6 0 27 2 EV 10693 13930 45 40 1 22 3 AA 12266 13303 34 36 1 22 1 B6 12478 14843 60 104 1 14 7 FL 10397 14635 1 -6 0 15 6 EV 12264 11042 57 40 1 18 7 OO 11292 13871 48 39 1 7 7 YV 10423 12264 87 92 1 21 1 DL 10397 11433 0 0 0 6 4 MQ 11193 11298 4 28 1 17 3 9E 14730 12478 -5 -7 0 23 3 OO 12266 11298 -1 -11 0 28 7 WN 14771 11292 30 35 1 11 5 WN 10721 12191 55 36 1 14 2 9E 13930 11193 -11 -23 0 20 5 AA 14771 12478 74 58 1 16 2 WN 14893 13891 5 8 0 23 2 EV 11292 13198 10 53 1 7 5 UA 13930 10721 27 25 0 17 1 DL 12478 12892 3 46 1 27 7 UA 12889 12266 7 15 0 28 2 AA 11298 12264 -2 -1 0 12 4 UA 14771 12266 0 27 1 24 3 WN 14679 14570 3 -3 0 21 6 9E 11193 13487 -11 -27 0 16 3 WN 11292 13871 19 11 0 12 1 WN 15016 13198 -2 -12 0 22 4 WN 14107 12892 60 62 1 15 1 WN 13232 12889 16 29 1 11 4 B6 14027 12953 -6 -24 0 27 6 EV 13930 11433 17 41 1 17 1 WN 10397 14730 148 134 1 19 3 US 14100 14771 63 75 1 18 7 WN 15016 11259 43 35 1 17 3 AA 11298 11278 112 145 1 7 5 EV 12266 11298 44 41 1 19 5 WN 10821 10423 17 15 0 15 7 AA 13495 11298 55 57 1 14 7 US 14107 13198 -4 1 0 3 4 B6 10721 12451 34 29 1 7 3 WN 13232 14307 7 61 1 24 1 US 12953 11057 -7 30 1 27 1 US 14100 12953 -4 -23 0 10 6 UA 12266 14107 7 -9 0 5 1 UA 12264 14747 34 55 1 14 6 DL 11278 10397 0 -5 0 28 1 AA 12953 13930 -8 1 0 24 5 AA 13303 11278 30 27 1 8 4 B6 10721 12892 10 40 1 19 4 AA 12478 12892 0 -11 0 4 4 B6 12478 15304 43 61 1 20 5 EV 13198 11618 99 78 1 17 3 YV 11057 12264 138 141 1 23 5 US 11278 14107 -4 -29 0 23 2 UA 12264 12892 -4 3 0 2 2 WN 15016 13232 6 -4 0 12 4 UA 14771 12264 96 67 1 29 1 WN 13487 13232 8 3 0 24 4 AS 14771 14057 47 46 1 20 1 FL 10397 11697 46 37 1 22 1 UA 12478 12892 -1 29 

Display the data as a bar chart to compare the distribution of the **Late** classes as you did previously. There should now be a more or less even number of each class. Then visualize the **DepDelay** field as a histogram and as a box plot to verify that the distribution, while still skewed, has fewer outliers.

### Explore Relationships in the Data
Predictive modeling is largely based on statistical relationships between fields in the data. To design a good model, you need to understand how the data points relate to one another.

A common way to start exploring relationships is to create visualizations that compare two or more data values. For example, modify the **Plot Options** of the chart above to compare the arrival delays for each carrier:
- **Keys**: Carrier
- **Series Groupings**: *none*
- **Values**: ArrDelay
- **Aggregation**: Count
- **Display Type**: Box plot

You may need to resize the plot to see the data clearly, but it should show that the median delay, and the distribution of delays varies by carrier; with some carriers having a higher median delay than others. The same is true for other features, such as the day of the week and the destination airport. You may already suspect that there's likely to be a relationship between delarture delay and arrival delay, so let's examine that next. Change the **Plot Options** as follows:
- **Keys**: None
- **Series Groupings**: *none*
- **Values**: ArrDelay, DepDelay
- **Aggregation**: Count
- **Display Type**: Scatter plot
- **Show LOESS**: Selected

The scatter plot shows the departure delay and corresponding arrival delay for each flight as a point in a two dimensional space. Note that the points form a diagonal line, which indicates a strong linear relationship between departure delay and arrival delay. This linear relationship shows a *correlation* between these two values, which we can measure statistically. The **corr** function calculates a correlation value between -1 and 1, indicating the strength of correlation between two fields. A strong positive correlation (near 1) indicates that high values for one column are often found with high values for the other, which a strong negative correlation (near -1) indicates that *low* values for one column are often found with *high* values for the other. A correlation near 0 indicates little apparent relationship between the fields.

In [32]:
resultValue = data.corr("DepDelay", "ArrDelay")

In this notebook we've cleaned the flight data, and explored it to identify some potential relationships between features of the flights and their lateness.

In [34]:
dbutils.notebook.exit(str(resultValue))

0.8969856340493185